In [ ]:
%pip install -r requirements.txt

In [1]:
import dagshub
import datetime
import mlflow
import torch
import time
import os
import shutil
import random
import yaml
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO, settings

In [ ]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-player-and-tennis-ball-ei0vm")
version = project.version(6)
dataset = version.download("yolov12", location="datasets/object-detection")

In [2]:
dagshub.init(repo_owner='abiyamf', repo_name='courtPlay', mlflow=True)
settings.update({"mlflow": True})

Accessing as nugrohorahmanto24

Initialized MLflow to track repo "abiyamf/courtPlay"

Repository abiyamf/courtPlay initialized!

In [3]:
# Global Config
os.environ["MLFLOW_KEEP_RUN_ACTIVE"] = "True"
time_now = datetime.datetime.now().strftime("%m-%d_%H-%M")
gpu_name = torch.cuda.get_device_name(0)
projectType = "object-detection"
datasetVersion = "6"
yoloVersion = "yolo12"
modelSize = "m"
epochs = 50
batch = -1
imgsz = 720
data_path = "datasets/object-detection/data.yaml"

def format_duration(seconds: float) -> str:
    """Convert seconds into 'Hh Mm Ss' format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}h {minutes}m {secs}s"

In [ ]:
model = YOLO(f"{yoloVersion}{modelSize}.pt")

mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

with mlflow.start_run(run_name=f"{time_now}"):
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
    })

    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=f"{time_now}",
        project=f"results/object-detection/{modelSize}",
        exist_ok=True
    )

    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })

    ## Tidak perlu karena sudah otomatis ter-log oleh ultralytics
    # mlflow.log_artifacts(f"results/object-detection/{modelSize}/{time_now}/weights/best.pt", artifact_path="best_model")

mlflow.end_run()